In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
properati = pd.read_csv('datos/caba_para_mapa.csv',index_col=0,error_bad_lines=False)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import validation_curve
from matplotlib.mlab import frange
from sklearn.preprocessing import StandardScaler  

properati=properati.dropna().reset_index()

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

<IPython.core.display.Javascript object>

In [2]:
def bucketnizer(elements_size,max_price,bucket_size=100):
    range_end=0
    bins=[]
    labels=[]
    for i in range(elements_size):
        bins.append(range_end)
        labels.append(i)
        range_end+=bucket_size
        if(range_end>max_price):
            break
    bins.append(max_price)
    return bins,labels

In [26]:
# Correme para bucketizar el set
clf = svm.SVC()
scaler = StandardScaler()  
max_price=properati.loc[properati['price_usd_per_m2'].idxmax()]['price_usd_per_m2']

# bins,labels=bucketnizer(len(properati.index),max_price)
bins,labels=bucketnizer(40,max_price)
print bins,labels
properati['categories_by_price']=pd.cut(properati['price_usd_per_m2'],bins,labels=labels)
properati['price_range']=pd.cut(properati['price_usd_per_m2'],bins)

[0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900, 3000, 3100, 3200, 3300, 3400, 3500, 3600, 3700, 3800, 3900, 5661.0] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]


In [15]:
clf = svm.SVC()
scaler = StandardScaler()  
cant_buckets = 15
#buckets
cantidad,rango = np.histogram(properati['price_usd_per_m2'], bins=cant_buckets)
properati['categories_by_price']=pd.cut(properati['price_usd_per_m2'],rango,labels=np.arange(cant_buckets))
properati['price_range']=pd.cut(properati['price_usd_per_m2'],rango)
# lo casteo a float porque si no tira error y tenia un nan nose porque
properati['categories_by_price']=properati['categories_by_price'].astype(np.float64) 
properati.dropna(inplace=True)

In [42]:
properati.groupby('categories_by_price').count()

,created_on,property_type,place_name,state_name,lat-lon,lat,lon,price,currency,price_aprox_local_currency,...,expenses,properati_url,description,title,dist_a_subte,dist_a_tren,dist_a_univ,dist_a_villa,dist_a_zona_anegada,price_range
categories_by_price,,,,,,,,,,,,,,,,,,,,,
0,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
1,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
2,43,43,43,43,43,43,43,43,43,43,...,43,43,43,43,43,43,43,43,43,43
3,186,186,186,186,186,186,186,186,186,186,...,186,186,186,186,186,186,186,186,186,186
4,579,579,579,579,579,579,579,579,579,579,...,579,579,579,579,579,579,579,579,579,579
5,677,677,677,677,677,677,677,677,677,677,...,677,677,677,677,677,677,677,677,677,677
6,452,452,452,452,452,452,452,452,452,452,...,452,452,452,452,452,452,452,452,452,452
7,261,261,261,261,261,261,261,261,261,261,...,261,261,261,261,261,261,261,261,261,261
8,122,122,122,122,122,122,122,122,122,122,...,122,122,122,122,122,122,122,122,122,122


# Validation Curve

In [ ]:
X = zip(properati['dist_a_subte'],properati['dist_a_univ'],properati['surface_total_in_m2'])
y = properati['categories_by_price']
scaler.fit_transform(X)

param_range=frange(1,10,1)
train_scores, test_scores = validation_curve(
clf, X, y, param_name="C",param_range=param_range,
cv=2,scoring="accuracy" ,n_jobs=-1)
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)

plt.title("Validation Curve with SVM")
plt.xlabel("Gamma")
plt.ylabel("Score")

plt.plot(param_range, train_scores_mean, label="Training score",
color="darkorange",marker="o")
plt.plot(param_range, test_scores_mean, label="Cross-validation score",
color="navy",marker="o")

plt.legend(loc="best")
plt.show()

# Random Search

In [17]:
import numpy as np
from time import time
from scipy.stats import randint as sp_randint

#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'])
y = properati['categories_by_price']

scaler.fit_transform(X,y)
#reviso si quedo datos anomalos
print np.any(np.isnan(y))
print np.all(np.isfinite(y))

False
True


In [34]:
C_range = np.arange(0.001,1000,0.001)
gamma_range = np.arange(0.001,1000,0.001)

In [35]:
%%notify
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedShuffleSplit

def report(results, n_top=10):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean training score: {0:.3f} (std: {1:.3f})".format(
                results['mean_train_score'][candidate],
                results['std_train_score'][candidate]))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                results['mean_test_score'][candidate],
                results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
    
param_grid ={"gamma": gamma_range, "C": C_range}
#cv = StratifiedShuffleSplit(n_splits=5, test_size=0.3) #Se cuelga si lo pones
random_search = RandomizedSearchCV(clf,param_distributions=param_grid,n_iter=50,cv=5,n_jobs=-1)
start = time()
random_search.fit(X, y)

print("RandomizedSearchCV took %.2f seconds for %d candidate parameter settings."
    % (time() - start, len(random_search.cv_results_['params'])))
report(random_search.cv_results_)

RandomizedSearchCV took 439.80 seconds for 50 candidate parameter settings.
Model with rank: 1
Mean training score: 0.724 (std: 0.003)
Mean validation score: 0.313 (std: 0.017)
Parameters: {'C': 183.839, 'gamma': 272.66499999999996}

Model with rank: 1
Mean training score: 0.724 (std: 0.003)
Mean validation score: 0.313 (std: 0.019)
Parameters: {'C': 5.9110000000000005, 'gamma': 228.119}

Model with rank: 1
Mean training score: 0.724 (std: 0.003)
Mean validation score: 0.313 (std: 0.017)
Parameters: {'C': 984.48500000000001, 'gamma': 274.28100000000001}

Model with rank: 1
Mean training score: 0.724 (std: 0.003)
Mean validation score: 0.313 (std: 0.017)
Parameters: {'C': 884.09299999999996, 'gamma': 280.21600000000001}

Model with rank: 5
Mean training score: 0.724 (std: 0.003)
Mean validation score: 0.312 (std: 0.012)
Parameters: {'C': 120.49400000000001, 'gamma': 114.38500000000001}

Model with rank: 5
Mean training score: 0.724 (std: 0.003)
Mean validation score: 0.312 (std: 0.012)


<IPython.core.display.Javascript object>